In [1]:
import pandas as pd
import datetime

# dataset
source: Kaggle | Quality Prediction in a Mining Process  
https://www.kaggle.com/datasets/edumagalhaes/quality-prediction-in-a-mining-process  
資料集取得於Kaggle上的Quality Prediction in a Mining Process的資料集，裡面包含時間跨度2017/03/10到2017/09/09泡沫浮選廠中鐵礦浮選流程中輸入到輸出，多數為每20秒產生一筆相關資訊，共24欄737,453筆。

In [2]:
df_mining_raw = pd.read_csv('./data/MiningProcess_Flotation_Plant_Database.csv')

In [3]:
# data type preprocess
for col in df_mining_raw.columns:
    if col != 'date':
        df_mining_raw[col] = df_mining_raw[col].str.replace(',', '.', regex=False)
        df_mining_raw[col] = pd.to_numeric(df_mining_raw[col])
    else:
        df_mining_raw[col] = pd.to_datetime(df_mining_raw[col]) 

In [5]:
df_mining_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 737453 entries, 0 to 737452
Data columns (total 24 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   date                          737453 non-null  datetime64[ns]
 1   % Iron Feed                   737453 non-null  float64       
 2   % Silica Feed                 737453 non-null  float64       
 3   Starch Flow                   737453 non-null  float64       
 4   Amina Flow                    737453 non-null  float64       
 5   Ore Pulp Flow                 737453 non-null  float64       
 6   Ore Pulp pH                   737453 non-null  float64       
 7   Ore Pulp Density              737453 non-null  float64       
 8   Flotation Column 01 Air Flow  737453 non-null  float64       
 9   Flotation Column 02 Air Flow  737453 non-null  float64       
 10  Flotation Column 03 Air Flow  737453 non-null  float64       
 11  Flotation Col

In [7]:
df_mining_raw['date'].min()

Timestamp('2017-03-10 01:00:00')

## date 補值
由於每筆資料為每20秒一筆，所以基本上每一個小時的資訊為180筆。而其中<日期時間>中未滿180筆，在此我們假設為該時間區段中前面的時間缺值，並利用backfill的方式，使用最近後面時間的資料補上缺失的時間區段。由於每筆資料為每20秒一筆，所以基本上每一個小時的資訊為180筆。而其中<日期時間>中未滿180筆，在此我們假設為該時間區段中前面的時間缺值，並利用backfill的方式，使用最近後面時間的資料補上缺失的時間區段。

In [4]:
temp = df_mining_raw.groupby('date').count()['% Iron Feed']
temp[temp < 180]

date
2017-03-10 01:00:00    174
2017-04-10 00:00:00    179
Name: % Iron Feed, dtype: int64

In [5]:
data_0310 = df_mining_raw[df_mining_raw['date']=='2017-03-10 01:00:00'].iloc[[0],:]
data_0410 = df_mining_raw[df_mining_raw['date']=='2017-04-10 00:00:00'].iloc[[0],:]

In [6]:
data_0310_all = pd.DataFrame()
for i in range(6):
    data_0310_all = pd.concat([data_0310_all, data_0310])

In [7]:
df0410 = df_mining_raw[df_mining_raw['date']=='2017-04-10 00:00:00']
df0410 = pd.concat([df0410, data_0410]).sort_index()

In [8]:
df0310 = df_mining_raw[df_mining_raw['date']=='2017-03-10 01:00:00']
df0310 = pd.concat([df0310, data_0310_all]).sort_index()

In [9]:
df_mining_tmp = df_mining_raw[~df_mining_raw['date'].isin(['2017-04-10 00:00:00', '2017-03-10 01:00:00'])]

In [10]:
df_dateclean = pd.concat([df_mining_tmp, df0310])
df_dateclean = pd.concat([df_dateclean, df0410])

In [11]:
df_dateclean = df_dateclean.sort_index()
df_dateclean = df_dateclean.reset_index(drop = True)

## date 切割20秒一筆
資料時間欄位的資料同一小時的資料標註時間皆相同，並沒有切割確切時間。所以我們以原始資料的順序進行標註，增加時間欄位datetime，讓每筆資料有獨立的時間標註。資料時間欄位的資料同一小時的資料標註時間皆相同，並沒有切割確切時間。所以我們以原始資料的順序進行標註，增加時間欄位datetime，讓每筆資料有獨立的時間標註。

In [12]:
all_date = df_dateclean.groupby('date').count().index

In [13]:
all_date_sc = []
for d in range(len(all_date)):
    for i in range(0, 180):
        all_date_sc.append(all_date[d] + datetime.timedelta(0,20*i))

In [14]:
df_dateclean['datetime'] = all_date_sc

## adjust y value df_dateclean
原始資料集在Kaggle的敘述中表示Y值也就是二氧化矽雜質為每一小時由實驗室出來的數據，所以同一小時的資訊應相同。然而有部分時間雜質比例確實不是如此，為20秒變動一次，我們懷疑為資料提供時有誤，所以我們將這些每小時內二氧化矽雜質不一致的資料進行每小時一次的平均，作為該小時代表之Y值。原始資料集在Kaggle的敘述中表示Y值也就是二氧化矽雜質為每一小時由實驗室出來的數據，所以同一小時的資訊應相同。然而有部分時間雜質比例確實不是如此，為20秒變動一次，我們懷疑為資料提供時有誤，所以我們將這些每小時內二氧化矽雜質不一致的資料進行每小時一次的平均，作為該小時代表之Y值。

In [15]:
iron_wrong_dates = df_dateclean.groupby(['date', '% Iron Concentrate']).count()[df_dateclean.groupby(['date', '% Iron Concentrate']).count()['% Iron Feed']==1].reset_index()['date']

In [16]:
iron_wrong_dates = iron_wrong_dates.drop_duplicates().to_list()

In [17]:
sil_wrong_dates = df_dateclean.groupby(['date', '% Silica Concentrate']).count()[df_dateclean.groupby(['date', '% Silica Concentrate']).count()['% Iron Feed']==1].reset_index()['date']

In [18]:
sil_wrong_dates = sil_wrong_dates.drop_duplicates().to_list()

In [19]:
all_wrong_date = sorted(list(set(sil_wrong_dates) | set(iron_wrong_dates)))

In [20]:
for d in all_wrong_date:
    this_iron_mean = round(df_dateclean[df_dateclean['date']==d]['% Iron Concentrate'].mean(), 2)
    this_sil_mean = round(df_dateclean[df_dateclean['date']==d]['% Silica Concentrate'].mean(), 2)
    df_dateclean.loc[df_dateclean["date"] == d, "% Iron Concentrate"] = this_iron_mean
    df_dateclean.loc[df_dateclean["date"] == d, "% Silica Concentrate"] = this_sil_mean

In [21]:
df_yclean = df_dateclean.copy()

In [22]:
df_yclean.shape

(737460, 25)

In [23]:
df_yclean.to_csv('./data/Flotation_Plant_preprocessed.csv', index = False)